In [1]:
import logging
import traceback

import pandas as pd
import xlwings as xw
from sixgill.definitions import *
from sixgill.pipesim import Model, Units

from network_simulation import NetworkSimulation, dict_to_df

# Constants
FOLDER_DIRECTORY = r"c:\Users\IDM252577\Desktop\Python Projects\PIPSIM"
MODEL_FILENAME = "BAB HP.pips"
EXCEL_FILE = "Water Injection Profile for BAB HP.xlsx"

SOURCE_NAME: str = "Sea Water Supply"
PUMP_NAME: list = ["HP Pump", "HP PUMP 2"]

ns = NetworkSimulation(FOLDER_DIRECTORY, MODEL_FILENAME, EXCEL_FILE)

for case in [ns.all_profiles.columns[0]]:
    for condition in [ns.all_conditions.index[0]]:
        try:
            ns.run_app(
                source_name=SOURCE_NAME,
                pump_name=PUMP_NAME,
                case=case,
                condition=condition,
                do_units_conversion=True,
            )
        except:
            traceback.print_exc()
            continue

INFO:network_simulation:------------Network Simulation Object Created----------------

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 13908
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:network_simulation:Model prepared
 Case:C1
 Condition: W-LP-EO
INFO:network_simulation:Set Global Conditions
INFO:network_simulation:Getting boundary conditions.....
INFO:network_simulation:Getting well values.....
INFO:network_simulation:Getting boundary conditions.....
INFO:network_simulation:Activated all wells
INFO:network_simulation:Deactivated no flow wells
INFO:network_simulation:Getting boundary conditions.....
INFO:network_simulation:Populated flowrates in model from excel
INFO:network_simulation:Model saved as c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\C1_W-LP-EO_BAB HP.pips
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_sim